In [235]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import warnings
warnings.filterwarnings("ignore")


## 1-) Veri Seti Oluşturma

In [236]:
#E-ticaret müşteri veri seti oluşturma
np.random.seed(42) 
n_customers=500 # 500 müşteri

data={
    #Numerik kolonlar
    "yas":np.random.randint(18,70,n_customers),
    "aylik_gelir":np.random.randint(22000,80000,n_customers),
    "harcama_skoru":np.random.randint(1,100,n_customers),
    "siparis_sayisi":np.random.randint(0,50,n_customers),
    "ortalama_sepet_tutari":np.random.uniform(50,500,n_customers),
    "siteyi_ziyaret_suresi_dk":np.random.exponential(15,n_customers),

    #Kategorik özellikler
    "cinsiyet":np.random.choice(["Erkek","Kadın"],n_customers),
    "sehir":np.random.choice(["Muğla","İzmir","İstanbul","Ankara","Balıkesir"],n_customers),
    "uyelik_tipi":np.random.choice(["Bronz","Silver","Altın","Platinium"],n_customers),
    'tercih_edilen_kategori': np.random.choice(['Elektronik', 'Giyim', 'Ev&Yaşam', 'Spor', 'Kitap'], n_customers),

    #Hedef değişken (sadakat programına katılım)
    "sadakat_programi":np.random.choice(["Evet","Hayır"],n_customers,p=[0.3,0.7])


}

In [237]:
df=pd.DataFrame(data)
df

,yas,aylik_gelir,harcama_skoru,siparis_sayisi,ortalama_sepet_tutari,siteyi_ziyaret_suresi_dk,cinsiyet,sehir,uyelik_tipi,tercih_edilen_kategori,sadakat_programi
0,56,25343,1,45,485.687043,6.877278,Erkek,İzmir,Platinium,Giyim,Evet
1,69,35500,5,22,133.486482,8.019854,Kadın,Ankara,Bronz,Giyim,Hayır
2,46,75222,12,31,440.880426,26.363666,Erkek,İstanbul,Bronz,Giyim,Hayır
3,32,51375,90,16,399.468584,19.842155,Kadın,Muğla,Platinium,Ev&Yaşam,Evet
4,60,31662,46,49,396.914830,21.999875,Erkek,İstanbul,Bronz,Spor,Hayır
...,...,...,...,...,...,...,...,...,...,...,...
495,65,59545,24,17,241.737523,28.684913,Kadın,Balıkesir,Silver,Spor,Evet
496,42,58668,81,14,108.819670,16.530483,Kadın,Ankara,Silver,Kitap,Evet
497,57,45061,74,43,448.971897,13.492300,Kadın,İzmir,Bronz,Kitap,Hayır
498,62,35716,30,15,252.403092,33.426433,Kadın,Ankara,Altın,Spor,Hayır


In [238]:
print(f"Veri seti boyutu {df.shape}")
print(f"\nÖzellikler: {df.columns.tolist()}")
print(f"\nVeri tipleri: {df.dtypes}")

Veri seti boyutu (500, 11)

Özellikler: ['yas', 'aylik_gelir', 'harcama_skoru', 'siparis_sayisi', 'ortalama_sepet_tutari', 'siteyi_ziyaret_suresi_dk', 'cinsiyet', 'sehir', 'uyelik_tipi', 'tercih_edilen_kategori', 'sadakat_programi']

Veri tipleri: yas                           int64
aylik_gelir                   int64
harcama_skoru                 int64
siparis_sayisi                int64
ortalama_sepet_tutari       float64
siteyi_ziyaret_suresi_dk    float64
cinsiyet                     object
sehir                        object
uyelik_tipi                  object
tercih_edilen_kategori       object
sadakat_programi             object
dtype: object


## 2-) Eksik veriler oluşturma

In [239]:
#Her sutun için eksik veri yüzdeleri
missing_percentages={
    "yas":0.05,
    "aylik_gelir":0.15,
    "harcama_skoru":0.10,
    "siparis_sayisi":0.08,
    "ortalama_sepet_tutari":0.12,
    "siteyi_ziyaret_suresi_dk":0.20,
    "cinsiyet":0.07,
    "sehir":0.10,
    "uyelik_tipi":0.05,
    "tercih_edilen_kategori":0.15
}

#Eksik verileri ekle
for col,missing_pct in missing_percentages.items():
    #rastgele indexler seç
    missing_indices=np.random.choice(df.index,size=int(len(df)*missing_pct),replace=False)

    #Bu indexlerin değerini "Nan" yap

    df.loc[missing_indices,col]=np.nan

print(df.isna().sum())

print("\nEksik veri yüzdesi")
print((df.isna().sum()/len(df)*100).round(2))

yas                          25
aylik_gelir                  75
harcama_skoru                50
siparis_sayisi               40
ortalama_sepet_tutari        60
siteyi_ziyaret_suresi_dk    100
cinsiyet                     35
sehir                        50
uyelik_tipi                  25
tercih_edilen_kategori       75
sadakat_programi              0
dtype: int64

Eksik veri yüzdesi
yas                          5.0
aylik_gelir                 15.0
harcama_skoru               10.0
siparis_sayisi               8.0
ortalama_sepet_tutari       12.0
siteyi_ziyaret_suresi_dk    20.0
cinsiyet                     7.0
sehir                       10.0
uyelik_tipi                  5.0
tercih_edilen_kategori      15.0
sadakat_programi             0.0
dtype: float64


## 3-) Veri ön işleme - SimpleImputer kullanımı-

In [240]:
#Numeric ve kategorik sutunları ayır
numeric_columns=["yas","aylik_gelir","harcama_skoru","siparis_sayisi",
                 "ortalama_sepet_tutari","siteyi_ziyaret_suresi_dk"]

categorical_columns=["cinsiyet","sehir","uyelik_tipi","tercih_edilen_kategori"]


### SimpleImputer ile eksik verileri doldurma

In [241]:
#Numeric sutunlar için SimpleImputer

#Stratejiler
print("Mean Stratejisi")
numeric_imputer_mean=SimpleImputer(strategy="mean")
df_numeric_mean=df[numeric_columns].copy()
df_numeric_mean_imputed=pd.DataFrame(numeric_imputer_mean.fit_transform(df_numeric_mean),columns=numeric_columns)

print("aylik_gelir  sütunu ile örnek")
print(f"Eksik veri sayısı(önce): {df["aylik_gelir"].isna().sum()}")
print(f"Ortalama değer: {df["aylik_gelir"].mean():.2f}")
print(f"Eksik veri sayisi(sonra): {df_numeric_mean_imputed["aylik_gelir"].isna().sum()}")

print("\nMedian Stratejisi")
numeric_imputer_median=SimpleImputer(strategy="median")
df_numeric_median=df[numeric_columns].copy()
df_numeric_median_imputed=pd.DataFrame(numeric_imputer_median.fit_transform(df_numeric_median),columns=numeric_columns)

print("harcama sütünu ile örnek")
print(f"Eksik veri sayısi(önce): {df["harcama_skoru"].isnull().sum()}")
print(f"Median değer: {df["harcama_skoru"].median():.2f}")
print(f"Eksik veri sayisi(sonra): {df_numeric_mean_imputed["harcama_skoru"].isna().sum()}")

#En Sık Stratejisi"
numeric_imputer_freq=SimpleImputer(strategy="most_frequent")
df_numeric_freq=df[numeric_columns].copy()
df_numeric_freq_imputed=pd.DataFrame(numeric_imputer_freq.fit_transform(df_numeric_freq),columns=numeric_columns)

#Sabit Değer Stratejisi
numeric_imputer_const=SimpleImputer(strategy="constant",fill_value=0)
df_numeric_const=df[numeric_columns].copy()
df_numeric_const_imputed=pd.DataFrame(numeric_imputer_const.fit_transform(df_numeric_const),columns=numeric_columns)

#En uygun strateji aykırı değerlere karşı daha dayanıklı olduğu için median
df[numeric_columns]=df_numeric_median_imputed


Mean Stratejisi
aylik_gelir  sütunu ile örnek
Eksik veri sayısı(önce): 75
Ortalama değer: 50632.16
Eksik veri sayisi(sonra): 0

Median Stratejisi
harcama sütünu ile örnek
Eksik veri sayısi(önce): 50
Median değer: 49.00
Eksik veri sayisi(sonra): 0


In [242]:
df.isna().sum()

yas                          0
aylik_gelir                  0
harcama_skoru                0
siparis_sayisi               0
ortalama_sepet_tutari        0
siteyi_ziyaret_suresi_dk     0
cinsiyet                    35
sehir                       50
uyelik_tipi                 25
tercih_edilen_kategori      75
sadakat_programi             0
dtype: int64

In [243]:
# Kategorik Sütünlar için SimpleImputer

print("En Sık Stratejisi")
categorical_imputer=SimpleImputer(strategy="most_frequent")
df_categorical=df[categorical_columns].copy()
df_categorical_imputed=pd.DataFrame(categorical_imputer.fit_transform(df_categorical),columns=categorical_columns)

print("Örnek olarak 'sehir' sütünu ")
print(f"Eksik veri sayısı(önce): {df["sehir"].isna().sum()}")
print(f"En sık değer: {df["sehir"].mode()[0] if not df["sehir"].mode().empty else 'N/A'}")
print(f"Eksik veri sayısı(sonra): {df_categorical_imputed["sehir"].isna().sum()}")

#Kategorik sütünları güncelle
df[categorical_columns]=df_categorical_imputed

print("\nTüm eksik veriler dolduruldu")
print("Eksik veri sayısi")
print("\n")
df.isna().sum()

En Sık Stratejisi
Örnek olarak 'sehir' sütünu 
Eksik veri sayısı(önce): 50
En sık değer: Muğla
Eksik veri sayısı(sonra): 0

Tüm eksik veriler dolduruldu
Eksik veri sayısi




yas                         0
aylik_gelir                 0
harcama_skoru               0
siparis_sayisi              0
ortalama_sepet_tutari       0
siteyi_ziyaret_suresi_dk    0
cinsiyet                    0
sehir                       0
uyelik_tipi                 0
tercih_edilen_kategori      0
sadakat_programi            0
dtype: int64

## 4-) Makine Öğrenmesi Modeli

### Kategorik Değişkenleri Encode Et

In [244]:
le_dict={}
for col in categorical_columns:
    le=LabelEncoder()
    df[col + "_encoded"]=le.fit_transform(df[col])
    le_dict[col]=le
    print(f"{col}: {dict(zip(le.classes_,le.transform(le.classes_)))}")

cinsiyet: {'Erkek': np.int64(0), 'Kadın': np.int64(1)}
sehir: {'Ankara': np.int64(0), 'Balıkesir': np.int64(1), 'Muğla': np.int64(2), 'İstanbul': np.int64(3), 'İzmir': np.int64(4)}
uyelik_tipi: {'Altın': np.int64(0), 'Bronz': np.int64(1), 'Platinium': np.int64(2), 'Silver': np.int64(3)}
tercih_edilen_kategori: {'Elektronik': np.int64(0), 'Ev&Yaşam': np.int64(1), 'Giyim': np.int64(2), 'Kitap': np.int64(3), 'Spor': np.int64(4)}


### Hedef değişkeni encode et

In [245]:
le_target=LabelEncoder()
y=le_target.fit_transform(df["sadakat_programi"])
print(f"Hedef değişken (sadakat programi): {dict(zip(le_target.classes_,le_target.transform(le_target.classes_)))}")

Hedef değişken (sadakat programi): {'Evet': np.int64(0), 'Hayır': np.int64(1)}


### Özellik matrisini oluştur

In [246]:
feature_colums=numeric_columns + [col +"_encoded" for col in categorical_columns]
X=df[feature_colums]

In [247]:
X

,yas,aylik_gelir,harcama_skoru,siparis_sayisi,ortalama_sepet_tutari,siteyi_ziyaret_suresi_dk,cinsiyet_encoded,sehir_encoded,uyelik_tipi_encoded,tercih_edilen_kategori_encoded
0,56.0,25343.0,1.0,45.0,485.687043,6.877278,0,4,2,2
1,69.0,49663.0,5.0,22.0,133.486482,8.019854,1,0,1,2
2,46.0,75222.0,12.0,31.0,440.880426,26.363666,0,3,1,2
3,32.0,51375.0,49.0,16.0,399.468584,10.257997,0,2,2,1
4,60.0,31662.0,49.0,49.0,396.914830,21.999875,0,3,1,1
...,...,...,...,...,...,...,...,...,...,...
495,65.0,59545.0,24.0,17.0,241.737523,28.684913,1,2,3,4
496,42.0,58668.0,81.0,14.0,108.819670,10.257997,1,0,3,3
497,57.0,45061.0,74.0,43.0,448.971897,13.492300,1,4,1,3
498,62.0,49663.0,30.0,15.0,252.403092,10.257997,0,0,0,4


In [248]:
print(f"Özellik matrisi boyutu: {X.shape}")
print(f"Hedef değişken boyutu: {y.shape}")

Özellik matrisi boyutu: (500, 10)
Hedef değişken boyutu: (500,)


In [249]:
#eğitim ve test setlerine ayır
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)
print(f"Eğitim seti: {X_train.shape}")
print(f"Test seti: {X_test.shape}")

Eğitim seti: (400, 10)
Test seti: (100, 10)


In [250]:
#Özellikleri ölçeklendir
scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

### RF modeli oluştur ve eğit

In [251]:
model=RandomForestClassifier(n_estimators=100,random_state=42,max_depth=10)
model.fit(X_train_scaled,y_train)

RandomForestClassifier(max_depth=10, random_state=42)

In [252]:
y_pred=model.predict(X_test_scaled)

### Model Performansı

In [253]:
accuracy=accuracy_score(y_test,y_pred)
print(f"Doğruluk Oranı: {accuracy}")

Doğruluk Oranı: 0.65


In [254]:
print("Sınıflandırma raporu")
print(classification_report(y_test,y_pred,target_names=le_target.classes_))

Sınıflandırma raporu
              precision    recall  f1-score   support

        Evet       0.38      0.09      0.15        33
       Hayır       0.67      0.93      0.78        67

    accuracy                           0.65       100
   macro avg       0.52      0.51      0.46       100
weighted avg       0.58      0.65      0.57       100



In [255]:
confusion_matrix(y_test,y_pred)

array([[ 3, 30],
       [ 5, 62]])